In [0]:
import collections
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras import models
from keras import layers


#train_df.groupby('class')['rating','class_freq'].mean().sort_values(by = 'class_freq', ascending=False)

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from keras.preprocessing.sequence import pad_sequences


from collections import Counter
import nltk
import seaborn as sns
import string
from nltk.corpus import stopwords
import re
#from autocorrect import spell
import re

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense
from keras.backend import eval
from keras.optimizers import Adam
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D,MaxPooling1D

In [2]:
from google.colab import files
uploaded_train = files.upload()
uploaded_test = files.upload()

Saving train.csv to train.csv


Saving test.csv to test.csv


In [0]:
import io
import pandas as pd

train = pd.read_csv(io.BytesIO(uploaded_train['train.csv']))
test = pd.read_csv(io.BytesIO(uploaded_test['test.csv']))

In [4]:
train.head()

,ID,Text,Class
0,7303,"""Please enable removing language code from the...",1
1,5258,"""Note: in your .csproj file, there is a Suppor...",0
2,6218,"""Wich means the new version not fully replaced...",0
3,3076,"""Some of my users will still receive the old x...",0
4,6508,"""The store randomly gives the old xap or the n...",0


In [5]:
test.head()

,ID,Text
0,9348,This would enable live traffic aware apps.
1,9393,Please try other formatting like bold italics ...
2,8557,Since computers were invented to save time I s...
3,9288,Allow rearranging if the user wants to change ...
4,9231,Add SIMD instructions for better use of ARM NE...


In [6]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
import string
from nltk.corpus import stopwords

stop=set(stopwords.words('english'))
def text_process(text):
    '''
    Takes in a string of text, then performs the following:
    1. Remove all stopwords
    2. Return the cleaned text as a list of words
    '''
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [0]:
def clean_document(doco):
    punctuation = string.punctuation
    punc_replace = ''.join([' ' for s in punctuation])
    doco_link_clean = re.sub(r'http\S+', '', doco)
    doco_clean_and = re.sub(r'&\S+', '', doco_link_clean)
    doco_clean_at = re.sub(r'@\S+', '', doco_clean_and)
    doco_clean = doco_clean_at.replace('-', ' ')
    doco_clean = doco_clean_at.replace('\r', ' ')
    doco_clean = doco_clean_at.replace('\n', ' ')
    doco_alphas = re.sub(r'\W +', ' ', doco_clean)
    trans_table = str.maketrans(punctuation, punc_replace)
    doco_clean = ' '.join([word.translate(trans_table) for word in doco_alphas.split(' ')])
    doco_clean = doco_clean.split(' ')
    p = re.compile(r'\s*\b(?=[a-z\d]*([a-z\d])\1{3}|\d+\b)[a-z\d]+', re.IGNORECASE)
    doco_clean = ([p.sub("", x).strip() for x in doco_clean])
    doco_clean = [word.lower() for word in doco_clean if len(word) > 2]
    doco_clean = ([i for i in doco_clean if i not in stop])
#     doco_clean = [spell(word) for word in doco_clean]
#     p = re.compile(r'\s*\b(?=[a-z\d]*([a-z\d])\1{3}|\d+\b)[a-z\d]+', re.IGNORECASE)
    doco_clean = ([p.sub("", x).strip() for x in doco_clean])
#     doco_clean = ([spell(k) for k in doco_clean])
    return doco_clean

In [9]:
suggestion_count = (train['Class'].values == 1).astype(int).sum()
non_suggestion_count = (train['Class'].values == 0).astype(int).sum()
print("Total sentences : " + str(train.shape[0]))
print("Total suggestions : " + str(suggestion_count))
print("Total non_suggestions : " + str(non_suggestion_count))

Total sentences : 8500
Total suggestions : 2085
Total non_suggestions : 6415


In [10]:
#Balancing the data

suggestions = train[train['Class'].values == 1]

while suggestion_count < non_suggestion_count:
    random_suggestion = suggestions.sample()
    train = train.append(random_suggestion, ignore_index = True)
    suggestion_count = suggestion_count + 1

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12830 entries, 0 to 12829
Data columns (total 3 columns):
ID       12830 non-null int64
Text     12830 non-null object
Class    12830 non-null int64
dtypes: int64(2), object(1)
memory usage: 300.8+ KB


In [11]:
import re
import numpy as np
from keras.preprocessing.text import Tokenizer

text_train = train['Text']
text_test = test['Text']

#clean the sentences
text_cleans_train = [clean_document(doc) for doc in text_train];
text_cleans_test = [clean_document(doc) for doc in text_test];

#join the list
sentences_train = [' '.join(r) for r in text_cleans_train ]
sentences_test = [' '.join(r) for r in text_cleans_test ]

sentences_all = sentences_train + sentences_test

#tokenize the sentences
tokenizer = Tokenizer(oov_token = 1)
tokenizer.fit_on_texts(sentences_all)
text_sequences = np.array(tokenizer.texts_to_sequences(sentences_train))
sequence_dict = tokenizer.word_index
word_dict = dict((num, val) for (val, num) in sequence_dict.items())
text_encoded = [];
for i,review in enumerate(text_cleans_train):
    text_encoded.append([sequence_dict[x] for x in review]);

Using TensorFlow backend.


In [12]:
len(text_cleans_train)

12830

In [0]:
NB_WORDS = 10000  # Parameter indicating the number of words we'll put in the dictionary
VAL_SIZE = 1000  # Size of the validation set
NB_START_EPOCHS = 10  # Number of epochs we usually start to train with
BATCH_SIZE = 512  # Size of the batches used in the mini-batch gradient descent
MAX_LEN = 100  # Maximum number of words in a sequence
GLOVE_DIM = 300  # Number of dimensions of the GloVe word embeddings

In [0]:
from keras.preprocessing.sequence import pad_sequences

X = pad_sequences(text_encoded, maxlen=MAX_LEN, truncating='post')

In [15]:
X

array([[   0,    0,    0, ...,  164,  701,  438],
       [   0,    0,    0, ...,   16, 1545, 3662],
       [   0,    0,    0, ...,  721, 1022,  448],
       ...,
       [   0,    0,    0, ...,  494,  193,   15],
       [   0,    0,    0, ...,   15,  138, 1164],
       [   0,    0,    0, ...,   63,  214,   19]], dtype=int32)

In [16]:
X.shape

(12830, 100)

In [17]:
! wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2020-01-19 19:11:29--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2020-01-19 19:11:29--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2020-01-19 19:11:29--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [18]:
! unzip 'glove.42B.300d.zip'

Archive:  glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [0]:
#using glove word embedding

glove_file = 'glove.42B.300d.txt'
emb_dict = {}
glove = open(glove_file)
for line in glove:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    emb_dict[word] = vector
glove.close()

In [0]:
emb_matrix = np.zeros((NB_WORDS, GLOVE_DIM))
for w, i in tokenizer.word_index.items():
    if i < NB_WORDS:
        vect = emb_dict.get(w)
        if vect is not None:
          emb_matrix[i] = vect
    else:
        break

In [21]:
#from keras.preprocessing.text import Tokenizer

# from keras.preprocessing.sequence import pad_sequences
# from keras.models import Sequential, Model
# from keras.layers import Input, Concatenate, GlobalMaxPooling1D, GlobalAveragePooling1D, Dense, Flatten, CuDNNLSTM, Conv1D, MaxPooling1D, Dropout, Activation, Bidirectional, CuDNNGRU, BatchNormalization, SpatialDropout1D
# from keras.layers.embeddings import Embedding

# units = 100

# inp = Input(shape = (MAX_LEN,))
# x = Embedding(NB_WORDS, GLOVE_DIM, input_length=MAX_LEN, weights=[emb_matrix], trainable=False)(inp)

# x1 = SpatialDropout1D(0.2)(x)

# x_gru = Bidirectional(CuDNNGRU(units, return_sequences = True))(x1)
# x1 = Conv1D(32, kernel_size=3, padding='valid', kernel_initializer='he_uniform')(x_gru)
# avg_pool1_gru = GlobalAveragePooling1D()(x1)
# max_pool1_gru = GlobalMaxPooling1D()(x1)

# x3 = Conv1D(32, kernel_size=2, padding='valid', kernel_initializer='he_uniform')(x_gru)
# avg_pool3_gru = GlobalAveragePooling1D()(x3)
# max_pool3_gru = GlobalMaxPooling1D()(x3)

# x_lstm = Bidirectional(CuDNNLSTM(units, return_sequences = True))(x1)
# x1 = Conv1D(32, kernel_size=3, padding='valid', kernel_initializer='he_uniform')(x_lstm)
# avg_pool1_lstm = GlobalAveragePooling1D()(x1)
# max_pool1_lstm = GlobalMaxPooling1D()(x1)

# x3 = Conv1D(32, kernel_size=2, padding='valid', kernel_initializer='he_uniform')(x_lstm)
# avg_pool3_lstm = GlobalAveragePooling1D()(x3)
# max_pool3_lstm = GlobalMaxPooling1D()(x3)


# x = Concatenate()([avg_pool1_gru, max_pool1_gru, avg_pool3_gru, max_pool3_gru,
#                 avg_pool1_lstm, max_pool1_lstm, avg_pool3_lstm, max_pool3_lstm])

# x = Dense(2, activation = "softmax")(x)
# model_glove1 = Model(inputs = inp, outputs = x)
# model_glove1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
#Another model

# from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Input, Concatenate, GlobalMaxPooling1D, GlobalAveragePooling1D, Dense, Flatten, CuDNNLSTM, Conv1D, MaxPooling1D, Dropout, Activation, Bidirectional, CuDNNGRU, BatchNormalization, SpatialDropout1D
from keras.layers.embeddings import Embedding
units = 100

inp = Input(shape = (MAX_LEN,))
x = Embedding(NB_WORDS, GLOVE_DIM, input_length=MAX_LEN, weights=[emb_matrix], trainable=False)(inp)
x = SpatialDropout1D(0.05)(x)
x = Bidirectional(CuDNNGRU(100, return_sequences = True))(x)
x = Conv1D(132, kernel_size=2 , padding='valid', kernel_initializer='he_uniform')(x)
avg_pool1 = GlobalAveragePooling1D()(x)
max_pool1 = GlobalMaxPooling1D()(x)
conc = Concatenate()([avg_pool1, max_pool1])
x = Dense(2, activation = "softmax")(conc)
model_glove1 = Model(inputs = inp, outputs = x)
model_glove1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
#Yet another model

# # from keras.preprocessing.text import Tokenizer

# from keras.preprocessing.sequence import pad_sequences
# from keras.models import Sequential
# from keras.layers import Dense, Flatten, CuDNNLSTM, Conv1D, MaxPooling1D, Dropout, Activation, Bidirectional, CuDNNGRU, BatchNormalization, SpatialDropout1D
# from keras.layers.embeddings import Embedding

# model_glove1 = Sequential()
# model_glove1.add(Embedding(NB_WORDS, GLOVE_DIM, input_length=MAX_LEN, weights=[emb_matrix], trainable=False))
# model_glove1.add(SpatialDropout1D(0.2))
# model_glove1.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
# model_glove1.add(Conv1D(128, 3, activation='relu'))
# model_glove1.add(Conv1D(128, 2, activation='relu'))
# model_glove1.add(MaxPooling1D(pool_size=6))
# model_glove1.add(CuDNNLSTM(100))
# model_glove1.add(Dense(2, activation='softmax'))
# model_glove1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# ## Fit train data
# #model_glove.fit(train['Text'], train['Class'], validation_split=0.4, epochs = 3)

In [0]:
import tensorflow as tf
Y = tf.keras.utils.to_categorical(train['Class'],num_classes=2)

In [0]:
from sklearn.model_selection import train_test_split

X_train_emb, X_valid_emb, y_train_emb, y_valid_emb = train_test_split(X, Y, test_size=0.2, random_state=37)

In [40]:
print(y_valid_emb.shape)

(2566, 2)


In [41]:
o = 0
z = 0

for i in range(len(y_valid_emb)):
  if (y_valid_emb[i][0] == 0):
    o = o+1
  else:
    z = z+1
print(o)
print(z)

1255
1311


In [42]:
from keras.models import load_model

#model_glove1=load_model("ang.h5")

filepath=r"model-2016B3A70340G_2.h5"
import keras
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', 
                                             verbose=1, save_best_only=True, mode='max')
model_glove1.fit(X_train_emb, y_train_emb, validation_data=(X_valid_emb, y_valid_emb), epochs=400 ,batch_size=1000, shuffle = True, callbacks=[checkpoint])
model_glove1.save(r"ang.h5")

Train on 10264 samples, validate on 2566 samples
Epoch 1/400
10264/10264 [==============================] - 3s 288us/step - loss: 0.6651 - acc: 0.5921 - val_loss: 0.5547 - val_acc: 0.7541

Epoch 00001: val_acc improved from -inf to 0.75409, saving model to model-2016B3A70340G_2.h5
Epoch 2/400
10264/10264 [==============================] - 1s 113us/step - loss: 0.5221 - acc: 0.7562 - val_loss: 0.4676 - val_acc: 0.7907

Epoch 00002: val_acc improved from 0.75409 to 0.79072, saving model to model-2016B3A70340G_2.h5
Epoch 3/400
10264/10264 [==============================] - 1s 114us/step - loss: 0.4525 - acc: 0.7948 - val_loss: 0.4331 - val_acc: 0.8118

Epoch 00003: val_acc improved from 0.79072 to 0.81177, saving model to model-2016B3A70340G_2.h5
Epoch 4/400
10264/10264 [==============================] - 1s 115us/step - loss: 0.4139 - acc: 0.8188 - val_loss: 0.4080 - val_acc: 0.8250

Epoch 00004: val_acc improved from 0.81177 to 0.82502, saving model to model-2016B3A70340G_2.h5
Epoch 5/40

In [43]:
from keras.models import load_model

filepath=r"model-2016B3A70340G_2.h5"
new=load_model(filepath)
new.summary()

score = new.evaluate(x=X_valid_emb , y=y_valid_emb)
print(score)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 100, 300)     3000000     input_2[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_3 (SpatialDro (None, 100, 300)     0           embedding_3[0][0]                
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) (None, 100, 200)     241200      spatial_dropout1d_3[0][0]        
____________________________________________________________________________________________

In [0]:
text_sequences_t = np.array(tokenizer.texts_to_sequences(sentences_test))
sequence_dict_t = tokenizer.word_index
word_dict_t = dict((num, val) for (val, num) in sequence_dict_t.items())
text_encoded_t = [];
for i,review in enumerate(text_cleans_test):
    text_encoded_t.append([sequence_dict_t[x] for x in review]);

In [0]:
max_cap = 100;
X_t = pad_sequences(text_encoded_t, maxlen=max_cap, truncating='post')

In [46]:
X_t

array([[   0,    0,    0, ..., 2316, 2026,    6],
       [   0,    0,    0, ..., 3009, 4877,   98],
       [   0,    0,    0, ...,  290,   71,  301],
       ...,
       [   0,    0,    0, ...,  578,  784,  142],
       [   0,    0,    0, ...,   85,  215,  253],
       [   0,    0,    0, ...,  556,  478,  324]], dtype=int32)

In [0]:
pred = new.predict(X_t)

In [0]:
pred_labels = np.argmax(pred,axis=1)
pred_labels.dtype
pred_labels = pred_labels.astype('int64')

In [0]:
sub = pd.DataFrame({  'ID' : test.ID ,'Class' : pred_labels})

In [50]:
sub.head()

,ID,Class
0,9348,0
1,9393,1
2,8557,0
3,9288,1
4,9231,1


In [0]:
sub.to_csv('exp1_3_12.csv', index = False)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
